<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/OpenMM/OpenMM_Scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**NOTE**: Code must be run twice to work

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()  # Will restart the runtime; re-run after restart

!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [2]:
!git clone https://github.com/Iourarum/GOPY.git
%cd GOPY
!python GOPY.py generate_PG 10 10 graphene.pdb
!python GOPY.py generate_GO graphene.pdb 2 4 8 functionalized.pdb

Cloning into 'GOPY'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 197 (delta 39), reused 18 (delta 18), pack-reused 150 (from 1)
Receiving objects: 100% (197/197), 9.50 MiB | 26.16 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/GOPY
done.
GOPY.py
generate_GO
graphene.pdb
2
4
8
functionalized.pdb
Left to add:  cooh:  2 epoxy:  4 hydroxyl:  8
Left to add:  cooh:  2 epoxy:  4 hydroxyl:  7
Left to add:  cooh:  1 epoxy:  4 hydroxyl:  7
Left to add:  cooh:  1 epoxy:  4 hydroxyl:  6
Left to add:  cooh:  1 epoxy:  4 hydroxyl:  5
Left to add:  cooh:  1 epoxy:  3 hydroxyl:  5
Left to add:  cooh:  1 epoxy:  2 hydroxyl:  5
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  5
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  4
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  3
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  2
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  1
Left to add:  cooh:  0 epoxy:  1

In [3]:
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/E1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/H1A.lib
!mkdir -p FRCMOD
!wget -P FRCMOD https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/FRCMOD/GO.frcmod

--2025-01-27 06:18:39--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1063 (1.0K) [text/plain]
Saving to: ‘GGG.lib’

GGG.lib             100%[===================>]   1.04K  --.-KB/s    in 0s      

2025-01-27 06:18:39 (55.2 MB/s) - ‘GGG.lib’ saved [1063/1063]

--2025-01-27 06:18:39--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1627 (1.6K) [text/plain]
Saving to: ‘C1A.lib’

C1

In [4]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 90.3 MB/s eta 0:00:00


In [ ]:
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picosecond, nanometer, picoseconds
import os, numpy as np, MDAnalysis as mda
from MDAnalysis.analysis.rdf import InterRDF
from scipy.signal import find_peaks
from scipy.spatial import cKDTree
from scipy.constants import Avogadro

def run_simulation_and_analyze(Na_count, box_dims, mass_adsorbent=1.0, initial_counts=None):
    # Create leap input for a given Na_count
    leap_content = f"""source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadoff GGG.lib
loadoff C1A.lib
loadoff E1A.lib
loadoff H1A.lib
loadamberparams FRCMOD/GO.frcmod
mol = loadpdb functionalized.pdb
bondbydistance mol
solvateBox mol TIP3PBOX 5.0
addIons2 mol Na+ {Na_count}
addIons2 mol Cl- {Na_count}
saveamberparm mol mol_solv.prmtop mol_solv.inpcrd
quit
"""
    with open('leap.in', 'w') as f:
        f.write(leap_content)
    os.system('tleap -f leap.in')

    # Run simulation
    prmtop = AmberPrmtopFile('mol_solv.prmtop')
    inpcrd = AmberInpcrdFile('mol_solv.inpcrd')
    topology = prmtop.topology
    positions = inpcrd.positions

    system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
    integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
    platform = Platform.getPlatformByName('CPU')
    simulation = Simulation(topology, system, integrator, platform)
    simulation.context.setPositions(positions)
    simulation.minimizeEnergy()
    simulation.reporters.append(PDBReporter('trajectory.pdb', 100))
    simulation.step(5000)

    # Analyze trajectory for Ce, qe, and RDF
    u = mda.Universe('mol_solv.prmtop', 'trajectory.pdb')
    for ts in u.trajectory:
        ts.dimensions = [box_dims[0], box_dims[1], box_dims[2], 90.0, 90.0, 90.0]
    u.trajectory.rewind()

    protein = u.select_atoms('resname GGG E1A H1A C1A')
    Na = u.select_atoms('name Na+')

    # RDF computation
    r_min, r_max = 0.0, 12.0
    rdf_calc = InterRDF(protein, Na, range=(r_min, r_max), nbins=120)
    rdf_calc.run()

    peaks, _ = find_peaks(rdf_calc.rdf)
    if len(peaks) > 0:
        first_peak = peaks[0]
        inverted = -rdf_calc.rdf
        minima, _ = find_peaks(inverted, distance=10)
        cands = minima[minima > first_peak]
        r_cut = rdf_calc.bins[cands[0]] if len(cands) > 0 else r_max
    else:
        r_cut = r_max

    adsorbed = 0
    total_frames = 0
    ref_group = protein
    target_group = Na
    for ts in u.trajectory:
        p_ref = ref_group.positions
        p_tg = target_group.positions
        tr_ref = cKDTree(p_ref)
        tr_tg = cKDTree(p_tg)
        sdm = tr_ref.sparse_distance_matrix(tr_tg, max_distance=r_cut, output_type='coo_matrix')
        col_ids = np.unique(sdm.col)
        adsorbed += len(col_ids)
        total_frames += 1

    avg_adsorbed = adsorbed / total_frames if total_frames > 0 else 0.0

    MW_Na = 22.99
    if initial_counts is None:
        initial_counts = {'Na': Na_count}
    N_initial = initial_counts['Na']

    volume_A3 = box_dims[0] * box_dims[1] * box_dims[2]
    volume_L = volume_A3 * 1e-24
    N_unads = N_initial - avg_adsorbed
    C_e = (N_unads * MW_Na / Avogadro) / volume_L * 1e3  # mg/L
    q_e = (avg_adsorbed * MW_Na) / mass_adsorbent * 1e3  # mg/g

    return C_e, q_e, rdf_calc.bins, rdf_calc.rdf

# Loop over different Na⁺ counts
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import csv

Na_counts = [20, 40, 60, 80]
box_dims = (150.0, 150.0, 100.0)
Ce_values = []
qe_values = []
rdf_data = []

for count in Na_counts:
    Ce, qe, r, rdf = run_simulation_and_analyze(count, box_dims, mass_adsorbent=1.0, initial_counts={'Na': count})
    Ce_values.append(Ce)
    qe_values.append(qe)
    rdf_data.append((count, r, rdf))
    print(f"Na_count: {count}, Ce: {Ce:.2f} mg/L, qe: {qe:.2f} mg/g")

# Save simulation data to CSV
with open('sorption_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Na_count', 'Ce (mg/L)', 'q_e (mg/g)'])
    for count, Ce, qe in zip(Na_counts, Ce_values, qe_values):
        writer.writerow([count, Ce, qe])

# Langmuir isotherm fitting and plotting
def langmuir(C, q_max, K_L):
    return (q_max * K_L * C) / (1 + K_L * C)

if len(Ce_values) > 2:
    params, _ = curve_fit(langmuir, Ce_values, qe_values, p0=[max(qe_values), 0.1], maxfev = 10000)
    q_max, K_L = params

    Ce_fit = np.linspace(min(Ce_values), max(Ce_values), 100)
    qe_fit = langmuir(Ce_fit, *params)

    plt.figure(figsize=(8, 6))
    plt.scatter(Ce_values, qe_values, c='b', label='Simulation Data')
    plt.plot(Ce_fit, qe_fit, 'r--', label='Langmuir Fit')
    plt.xlabel('Cₑ (mg/L)')
    plt.ylabel('qₑ (mg/g)')
    plt.title('Na⁺ Sorption Isotherm on Activated Carbon')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print(f"Langmuir fit parameters:\nq_max = {q_max:.2f} mg/g\nK_L = {K_L:.4f} L/mg")

# RDF plotting for each trial
plt.figure(figsize=(10, 6))
for count, r, rdf in rdf_data:
    plt.plot(r, rdf, label=f'Na_count={count}')
plt.xlabel('Distance r (Å)')
plt.ylabel('g(r)')
plt.title('Radial Distribution Function (RDF) for Each Na⁺ Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

/usr/local/lib/python3.11/site-packages/MDAnalysis/analysis/base.py:522: UserWarning: Reader has no dt information, set to 1.0 ps
  self.times[idx] = ts.time
/usr/local/lib/python3.11/site-packages/MDAnalysis/analysis/rdf.py:347: DeprecationWarning: The `rdf` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.rdf` instead
  warnings.warn(wmsg, DeprecationWarning)
/usr/local/lib/python3.11/site-packages/MDAnalysis/analysis/rdf.py:339: DeprecationWarning: The `bins` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.bins` instead
  warnings.warn(wmsg, DeprecationWarning)


Na_count: 20, Ce: 0.31 mg/L, qe: 44600.60 mg/g


In [8]:
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds, StateDataReporter
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picosecond, nanometer, picoseconds
import os
import numpy as np

def run_simulation_and_generate_outputs(Na_count, simulation_steps=5000):
    # Step 1: Create leap input for a given Na_count
    leap_content = f"""source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadoff GGG.lib
loadoff C1A.lib
loadoff E1A.lib
loadoff H1A.lib
loadamberparams FRCMOD/GO.frcmod
mol = loadpdb functionalized.pdb
bondbydistance mol
solvateBox mol TIP3PBOX 5.0
addIons2 mol Na+ {Na_count}
addIons2 mol Cl- {Na_count}
saveamberparm mol mol_solv.prmtop mol_solv.inpcrd
quit
"""
    with open('leap.in', 'w') as f:
        f.write(leap_content)

    # Step 2: Run tleap to prepare the system
    !tleap -f leap.in > leap.log

    # Check if tleap ran successfully
    if not os.path.exists('mol_solv.prmtop') or not os.path.exists('mol_solv.inpcrd'):
        raise FileNotFoundError("tleap failed to create parameter/topology files. Check leap.log for details.")

    # Step 3: Set up OpenMM simulation
    prmtop = AmberPrmtopFile('mol_solv.prmtop')
    inpcrd = AmberInpcrdFile('mol_solv.inpcrd')
    topology = prmtop.topology
    positions = inpcrd.positions

    system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
    integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
    platform = Platform.getPlatformByName('CPU')  # Use 'CUDA' or 'OpenCL' if GPU is available
    simulation = Simulation(topology, system, integrator, platform)
    simulation.context.setPositions(positions)

    # Minimize energy
    simulation.minimizeEnergy()

    # Add reporters to save trajectory and log data
    simulation.reporters.append(PDBReporter('trajectory.pdb', 1000))  # Save every 1000 steps
    simulation.reporters.append(StateDataReporter('md1.out', 1000, step=True,
                                                  potentialEnergy=True, temperature=True,
                                                  totalEnergy=True, density=True))

    # Run the simulation
    simulation.step(simulation_steps)

    # For energy equilibrium, generate Eq2.1.out (assuming this is required)
    # For demonstration, we'll copy md1.out to Eq2.1.out
    # In practice, Eq2.1.out should be generated based on your simulation specifics
    !cp md1.out Eq2.1.out

# Example usage:
# Adding 10 Na+ ions; adjust Na_count as needed
run_simulation_and_generate_outputs(Na_count=10, simulation_steps=5000)


In [9]:
# Extract density values from md1.out
!awk '/Density/ { print $3 }; $1=="A" && $2=="V" {exit 0}' md1.out > density.dat
# Calculate average density
!awk '{ sum += $1 } END { print "average = ", sum/NR }' density.dat
# Extract total energy values from Eq2.1.out
!awk '/Etot/ {print $3}; ($1=="A" && $2=="V") {exit 0};' Eq2.1.out > etot.dat
# Calculate average total energy
!awk '{ sum += $1 } END { print "average total energy = ", sum/NR }' etot.dat
import matplotlib.pyplot as plt

# Plot Density Data
density = np.loadtxt('density.dat')

plt.figure(figsize=(10, 5))
plt.plot(density, label='Density')
plt.xlabel('Time Steps')
plt.ylabel('Density (units)')
plt.title('Density Over Time')
plt.legend()
plt.grid(True)
plt.show()

# Plot Energy Total Data
etot = np.loadtxt('etot.dat')

plt.figure(figsize=(10, 5))
plt.plot(etot, label='Total Energy', color='orange')
plt.xlabel('Time Steps')
plt.ylabel('Total Energy (units)')
plt.title('Total Energy Over Time')
plt.legend()
plt.grid(True)
plt.show()

average =  0
average total energy =  -nan


ValueError: could not convert string '(kJ/mole)","Total' to float64 at row 0, column 1.

In [12]:
# Create ptraj.in file
ptraj_in_content = """trajin trajectory.pdb
radial Protein_Na .1 15.0 :GGG,E1A,H1A,C1A@N :Na+
"""

with open('ptraj.in', 'w') as f:
    f.write(ptraj_in_content)
# Run ptraj to calculate RDFs
!ptraj mol_solv.prmtop < ptraj.in > ptraj.log

/bin/bash: line 1: ptraj: command not found


In [10]:
!awk '/Density/ { print $3 }; $1=="A" && $2=="V" {exit 0}' md1.out > density.dat

In [11]:
!awk '{ sum += $1 } END { print "average = ", sum/NR }' density.dat

average =  0
